## Notebook for illustrating kernel search

Define search space and initialize kernel search wrapper 

In [ ]:
from alef.configs.models.gp_model_kernel_search_config import (
    GPBigLocalNDimFullKernelSearchConfig,
)

from alef.models.model_factory import ModelFactory
import logging

logging.basicConfig(level=logging.INFO)
##### Settings #######
# configure path where all datasets lie
dataset_base_path = ""
n_train = 150
# kernel search will last around 10-20 min depending on your hardware
model_config = GPBigLocalNDimFullKernelSearchConfig(input_dimension=2)
model_config.n_steps_bo: int = 25
model_config.fast_inference = True
model = ModelFactory.build(model_config)

Load LGBB Dataset (useful to show kernel search)

In [ ]:
from alef.data_sets.lgbb import LGBB

dataset = LGBB(base_path=dataset_base_path)
dataset.load_data_set()
x_train, y_train, x_test, y_test = dataset.sample_train_test(True, n_train, 400, 0.8)

Start kernel search via calling model.infer

In [ ]:
model.infer(x_train, y_train)

In [ ]:
from alef.utils.plotter2D import Plotter2D
import numpy as np

pred_mu_test, pred_sigma_test = model.predictive_dist(x_test)

plotter_object = Plotter2D(3)
plotter_object.add_gt_function(x_test, np.squeeze(pred_sigma_test), "RdBu_r", 14, 0)
plotter_object.add_datapoints(x_train, "black", 0)
min_y = np.min(y_test)
max_y = np.max(y_test)
levels = np.linspace(min_y, max_y, 100)
plotter_object.add_gt_function(x_test, np.squeeze(y_test), "seismic", levels, 1)
plotter_object.add_datapoints(x_train, "black", 1)
plotter_object.add_gt_function(x_test, np.squeeze(pred_mu_test), "seismic", levels, 2)
plotter_object.add_datapoints(x_train, "black", 2)
plotter_object.show()